In [46]:
import pandas as pd
import numpy as np
import mosaic.io as mio
import plotly.express as px
from pathlib import Path
import os
import seaborn as sns
import itertools

from scipy.stats import betabinom
from scipy.stats import binom
from scipy.special import logsumexp
from scipy.stats import fisher_exact

from poibin import PoiBin

In [2]:
patient_name = "PC11"
pt_h5 = f"../data/rebuttal/{patient_name}.patient_wide.genotyped.h5"

In [3]:
pt = mio.load(pt_h5)

Loading, ../data/rebuttal/PC11.patient_wide.genotyped.h5


/Users/psashittal/Documents/research/mosaic/src/mosaic/assay.py:218: UserWarning: Multiple samples detected. Appending the sample name to the barcodes to avoid duplicate barcodes. The original barcodes are stored in the original_barcode row attribute
  warnings.warn(msg)


Loaded in 1.3s.


In [4]:
cravat = pd.read_csv(f"../data/rebuttal/{patient_name}_CRAVAT_output_cleaned.txt", sep = "\t", index_col = 0, header=[0,1])

In [5]:
cravat = cravat[
        (~cravat[("Variant Annotation", "Sequence Ontology")].isin([])) & 
        (~(cravat[("PoN_comparison", "PoN-superset-normals-occurence")] > 1))
        ]

In [6]:
pt.dna.genotype_variants()

In [7]:
snv_f = f"../data/rebuttal/{patient_name}-patient-all_vars-voi.hz_curated.txt"

In [8]:
snv_df = pd.read_csv(snv_f, sep = '\t', index_col = 0, comment='#')
snv_df["annotation"].fillna("", inplace=True)
# filter out artifact
snv_df = snv_df[~snv_df['annotation'].str.contains("artifact")]
# filter out germline HOM
snv_df = snv_df[~snv_df['annotation'].str.contains("germline_HOM")]
snv_df.sort_values(by=["mut_prev", "HGVSp"], ascending=False)
snv_ann_map = snv_df['HGVSp'].to_dict()

In [9]:
VOI = snv_df[snv_df["annotation"].str.contains("somatic")].index.tolist()
VOI += ["chr12:25398284:C/T"]
VOI = [v for v in VOI if v != "chr3:178936082:G/A"]
snv_ann_map["chr12:25398284:C/T"] = "KRAS p.Gly12Asp"

In [10]:
df_total = pt.dna.get_attribute('DP', constraint='row', features=VOI)

In [11]:
df_alt = pt.dna.get_attribute('alt_read_count', constraint='row', features=VOI)

# current mutual exclusivity test

In [12]:
ncells = df_alt.shape[0]
nmutations = df_alt.shape[1]
mutation_list = list(df_alt.columns)

In [13]:
ado_precision = 15 # precision parameter
fp = 0.001 # false positive rate

In [14]:
# this part needs work -- how do you go from read counts to probabilities of presence of mutations?
total_reads_mat = df_total.values
alt_reads_mat = df_alt.values

bb_alpha = fp * ado_precision
bb_beta = (1 - fp) * ado_precision

presence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, 1, 1)
absence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, bb_alpha, bb_beta)

presence_absence_tensor = np.stack([presence_coeff_mat, absence_coeff_mat], axis=2)
total_coeff_mat = logsumexp(presence_absence_tensor, axis=2)

normalized_presence_prob = np.exp(presence_coeff_mat - total_coeff_mat)

In [ ]:
# presence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, 1, 1)
# absence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, bb_alpha, bb_beta)

In [28]:
normalized_presence_prob[df_alt == 0] = 0

In [29]:
# # alternative probability model
# vaf_threshold = 0.1
# dp_threshold = 10
# alt_threshold = 3

# df_vaf = df_alt / df_total

# df_mutation = ((df_vaf >= vaf_threshold) & (df_total >= dp_threshold) & (df_alt >= alt_threshold)).astype(int)

# weight_presence = 0.9
# weight_absence = 0.1
# df_prob = df_mutation * (weight_presence - weight_absence) + weight_absence

# normalized_presence_prob = df_prob.values

In [31]:
# g = sns.clustermap(normalized_presence_prob, col_cluster=False, figsize=(8,6))
# # ax.set_xticklabels(df_total.columns, rotation=90);
# g.ax_heatmap.axes.set_xticklabels(df_alt.columns, rotation=90);

In [32]:
null_prob_vector = np.exp(logsumexp(np.log(normalized_presence_prob), axis=0) - np.log(ncells))

/var/folders/r8/hw328ck96hxbslq1ftmyz7tr0000gn/T/ipykernel_30996/1457504995.py:1: RuntimeWarning: divide by zero encountered in log
  null_prob_vector = np.exp(logsumexp(np.log(normalized_presence_prob), axis=0) - np.log(ncells))


In [33]:
null_prob_vector

array([0.42373204, 0.36158246, 0.35912074, 0.0194819 ])

In [34]:
test_data =[]
for mut1_idx, mut2_idx in itertools.combinations(range(nmutations), 2):

    p_vector = (normalized_presence_prob[:, mut1_idx] * (1 - normalized_presence_prob[:, mut2_idx]) +
                normalized_presence_prob[:, mut2_idx] * (1 - normalized_presence_prob[:, mut1_idx]))

    null_prob = (null_prob_vector[mut1_idx] * (1 - null_prob_vector[mut2_idx]) +
                 null_prob_vector[mut2_idx] * (1 - null_prob_vector[mut1_idx]))

    pb = PoiBin(p_vector)

    me_inv_pval = 0
    for k in range(ncells):
        me_inv_pval += pb.pval(k) * binom.pmf(k, ncells, null_prob)
        
    test_data.append([mutation_list[mut1_idx], mutation_list[mut2_idx], 1 - me_inv_pval])

In [35]:
df_test = pd.DataFrame(test_data, columns = ['mutation1', 'mutation2', 'exclusivity'])

In [38]:
df_test

,mutation1,mutation2,exclusivity
0,chr12:25398284:C/A,chr18:48573531:G/A,1.000000
1,chr12:25398284:C/A,chr18:48573570:G/A,1.000000
2,chr12:25398284:C/A,chr12:25398284:C/T,0.968169
3,chr18:48573531:G/A,chr18:48573570:G/A,1.000000
4,chr18:48573531:G/A,chr12:25398284:C/T,0.963774
5,chr18:48573570:G/A,chr12:25398284:C/T,0.961625


In [23]:
df_test

,mutation1,mutation2,exclusivity
0,chr12:25398284:C/A,chr18:48573531:G/A,1.000000
1,chr12:25398284:C/A,chr18:48573570:G/A,1.000000
2,chr12:25398284:C/A,chr12:25398284:C/T,0.592880
3,chr18:48573531:G/A,chr18:48573570:G/A,1.000000
4,chr18:48573531:G/A,chr12:25398284:C/T,0.176813
5,chr18:48573570:G/A,chr12:25398284:C/T,0.171628


In [22]:
df_test[df_test['exclusivity'] < 0.1]

,mutation1,mutation2,exclusivity


In [187]:
df_test[df_test['exclusivity'] < 0.05]

,mutation1,mutation2,exclusivity
3,chr2:25457199:T/C,chr16:347184:A/G,3.668695e-03
4,chr2:25457199:T/C,chr19:11096851:A/G,2.393277e-03
5,chr2:25457199:T/C,chr17:29685905:A/G,2.622583e-04
7,chr2:25457199:T/C,chr1:27099479:G/A,2.086475e-04
9,chr17:7577556:C/T,chr17:41245466:G/A,2.635669e-13
21,chr17:41245466:G/A,chr16:347184:A/G,3.381742e-04
23,chr17:41245466:G/A,chr17:29685905:A/G,3.270965e-04
24,chr17:41245466:G/A,chr12:49440207:C/A,1.419448e-02
25,chr17:41245466:G/A,chr1:27099479:G/A,2.172815e-03


# using TEA

In [25]:
from tea.snv.me import get_exclusivity

In [26]:
E = 1 - get_exclusivity(
    pt.dna.get_attribute('DP', constraint='row', features=VOI),
    pt.dna.get_attribute('alt_read_count', constraint='row', features=VOI),
    # rm_irrelevant_cells = False
)

Time to compute presence probabilities: 0.024572134017944336


/Users/psashittal/Documents/research/tea/src/tea/snv/me.py:112: RuntimeWarning: divide by zero encountered in log
  null_prob_mut_1 = np.exp(logsumexp(np.log(npp_mut_1), axis=0) - np.log(ncells))
/Users/psashittal/Documents/research/tea/src/tea/snv/me.py:113: RuntimeWarning: divide by zero encountered in log
  null_prob_mut_2 = np.exp(logsumexp(np.log(npp_mut_2), axis=0) - np.log(ncells))


Time to compute mutual exclusivity: 3.539987087249756


In [27]:
E

,chr12:25398284:C/A,chr18:48573531:G/A,chr18:48573570:G/A,chr12:25398284:C/T
chr12:25398284:C/A,1.000000e+00,1.000000e+00,1.000000e+00,4.681577e-07
chr18:48573531:G/A,1.000000e+00,1.000000e+00,1.000000e+00,1.305844e-12
chr18:48573570:G/A,1.000000e+00,1.000000e+00,1.000000e+00,1.451506e-12
chr12:25398284:C/T,4.681577e-07,1.305844e-12,1.451506e-12,1.000000e+00


# Fisher exact test

In [122]:
vaf_threshold = 0.1
dp_threshold = 10
alt_threshold = 3

In [123]:
df_vaf = df_alt / df_total

In [124]:
df_mutation = ((df_vaf >= vaf_threshold) & (df_total >= dp_threshold) & (df_alt >= alt_threshold)).astype(int)

In [125]:
df_mutation

,chr12:25398284:C/A,chr18:48573531:G/A,chr18:48573570:G/A,chr12:25398284:C/T
alt_read_count,,,,
cell_1-RA17_22-32_1,0,0,0,0
cell_10-RA17_22-32_1,0,0,0,0
cell_100-RA17_22-32_1,0,0,0,0
cell_1000-RA17_22-32_1,0,0,0,0
cell_1001-RA17_22-32_1,0,0,0,0
...,...,...,...,...
cell_95-RA17_22-27_2,0,0,0,0
cell_96-RA17_22-27_2,0,0,0,0
cell_97-RA17_22-27_2,1,0,1,0


In [126]:
df_mutation2 = df_mutation.copy()

In [113]:
mutation_list = df_mutation.columns

In [87]:
test_data = []
for mutation1, mutation2 in itertools.combinations(mutation_list, 2):
    # print(mutation1, mutation2)
    
    confusion_mat = np.zeros((2,2))
    
    vec1 = df_mutation[mutation1].values
    vec2 = df_mutation[mutation2].values
    
    confusion_mat[0,0] = ((vec1 == 1) & (vec2 == 1)).sum()
    confusion_mat[1,0] = ((vec1 == 0) & (vec2 == 1)).sum()
    confusion_mat[0,1] = ((vec1 == 1) & (vec2 == 0)).sum()
    confusion_mat[1,1] = ((vec1 == 0) & (vec2 == 0)).sum()    
    
    exclusivity_pvalue = fisher_exact(confusion_mat, alternative='less')[1]
    co_occurrence_pvalue = fisher_exact(confusion_mat, alternative='greater')[1]
    
    test_data.append([mutation1, mutation2, exclusivity_pvalue, co_occurrence_pvalue])

In [88]:
df_test = pd.DataFrame(test_data, columns = ['mutation1', 'mutation2', 'exclusivity', 'co-occurrence'])

In [89]:
df_test

,mutation1,mutation2,exclusivity,co-occurrence
0,chr12:25398284:C/A,chr18:48573531:G/A,1.000000,0.000000
1,chr12:25398284:C/A,chr18:48573570:G/A,1.000000,0.000000
2,chr12:25398284:C/A,chr12:25398284:C/T,0.015592,0.995188
3,chr18:48573531:G/A,chr18:48573570:G/A,1.000000,0.000000
4,chr18:48573531:G/A,chr12:25398284:C/T,0.043613,0.985552
5,chr18:48573570:G/A,chr12:25398284:C/T,0.044521,0.985199


# fisher exact test with likelihood-based genotyping

In [134]:
ado_precision = 15 # precision parameter
fp = 0.001 # false positive rate

In [135]:
total_reads_mat = df_total.values
alt_reads_mat = df_alt.values

bb_alpha = fp * ado_precision
bb_beta = (1 - fp) * ado_precision

presence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, 1, 1)
absence_coeff_mat = betabinom.logpmf(alt_reads_mat, total_reads_mat, bb_alpha, bb_beta)

In [136]:
df_mutation = pd.DataFrame((presence_coeff_mat > absence_coeff_mat).astype(int), 
                           index=df_total.index,
                           columns=df_total.columns)

df_mutation[df_total < 10] = 0

In [137]:
mutation_list = df_mutation.columns

In [138]:
test_data = []
for mutation1, mutation2 in itertools.combinations(mutation_list, 2):
    # print(mutation1, mutation2)
    
    confusion_mat = np.zeros((2,2))
    
    vec1 = df_mutation[mutation1].values
    vec2 = df_mutation[mutation2].values
    
    confusion_mat[0,0] = ((vec1 == 1) & (vec2 == 1)).sum()
    confusion_mat[1,0] = ((vec1 == 0) & (vec2 == 1)).sum()
    confusion_mat[0,1] = ((vec1 == 1) & (vec2 == 0)).sum()
    confusion_mat[1,1] = ((vec1 == 0) & (vec2 == 0)).sum()    
    
    exclusivity_pvalue = fisher_exact(confusion_mat, alternative='less')[1]
    co_occurrence_pvalue = fisher_exact(confusion_mat, alternative='greater')[1]
    
    test_data.append([mutation1, mutation2, exclusivity_pvalue, co_occurrence_pvalue])

In [139]:
df_test = pd.DataFrame(test_data, columns = ['mutation1', 'mutation2', 'exclusivity', 'co-occurrence'])

In [150]:
df_test

,mutation1,mutation2,exclusivity,co-occurrence
0,chr12:25398284:C/A,chr18:48573531:G/A,1.000000,0.000000e+00
1,chr12:25398284:C/A,chr18:48573570:G/A,1.000000,0.000000e+00
2,chr12:25398284:C/A,chr12:25398284:C/T,1.000000,2.642875e-08
3,chr18:48573531:G/A,chr18:48573570:G/A,1.000000,0.000000e+00
4,chr18:48573531:G/A,chr12:25398284:C/T,0.999999,2.147331e-06
5,chr18:48573570:G/A,chr12:25398284:C/T,0.999998,4.610121e-06
